In [164]:
import cv2
import numpy as np

In [165]:
pair = cv2.cvtColor(cv2.imread("sig_rabbit_pair_0000.exr", cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYDEPTH), cv2.COLOR_BGR2RGB)
ref = cv2.imread("sig_rabbit_shot_0000.exr", cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYDEPTH)
src = cv2.imread("sig_rabbit_shot_0001.exr", cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYDEPTH)

In [166]:
pair[:,:,:2].mean(), pair[:,:,:2].std()

(-1.716232, 16.428417)

In [167]:
h, w, nc = ref.shape
assert src.shape == (h, w, nc), "should be same size"

In [168]:
corr = np.zeros((2*h, w, nc))
corr[:h,:,:] = ref
corr[h:,:,:] = src

N_SAMPLES = 15
valid = np.where(pair[:,:,2] == 1)
valid_n = len(valid[1])

for _ in range(N_SAMPLES):
    r = np.random.random()
    g = np.random.random()
    b = np.random.random()
    
    idx = np.random.randint(0, valid_n)
    p = (valid[0][idx], valid[1][idx])
        
    info = pair[p]
    if info[2] == 0:
        continue
    
    corr = cv2.circle(corr, (p[1], p[0]), 5, (b,g,r), 2)
    
    # ------- y ------x ----
    flow = (info[1], info[0])
    src_p = (p[0] + flow[0], p[1] + flow[1])
    if src_p[0] < 0 or src_p[0] >= h or src_p[1] < 0 or src_p[1] >= w:
        continue
    corr = cv2.circle(corr, (int(src_p[1]), int(src_p[0])+h), 5, (b,g,r), 2)
    corr = cv2.line(corr, (p[1], p[0]), (int(src_p[1]), int(src_p[0])+h), (b,g,r), 2)
    print(p, src_p, info)
    
cv2.imshow("x", corr)
cv2.waitKey()
cv2.destroyAllWindows()

(270, 632) (286.796875, 563.25) [-68.75      16.796875   1.      ]
(257, 670) (297.1875, 594.5) [-75.5     40.1875   1.    ]
(188, 537) (128.46875, 444.5) [-92.5     -59.53125   1.     ]
(408, 626) (439.265625, 509.875) [-116.125      31.265625    1.      ]
(391, 536) (358.0625, 452.625) [-83.375  -32.9375   1.    ]
(333, 615) (350.375, 503.375) [-111.625   17.375    1.   ]
(445, 607) (461.078125, 490.1875) [-116.8125     16.078125    1.      ]
(514, 623) (537.578125, 511.0) [-112.         23.578125    1.      ]
(453, 668) (519.5625, 570.875) [-97.125   66.5625   1.    ]
(372, 661) (429.28125, 559.0625) [-101.9375    57.28125    1.     ]
(265, 659) (299.5625, 585.875) [-73.125   34.5625   1.    ]
(313, 660) (360.53125, 562.25) [-97.75     47.53125   1.     ]
(176, 548) (117.53125, 457.5625) [-90.4375  -58.46875   1.     ]
(403, 689) (487.125, 608.3125) [-80.6875  84.125    1.    ]
(332, 681) (399.875, 619.1875) [-61.8125  67.875    1.    ]


In [169]:
# cv2.imwrite("rabbit_corr.png",255*corr)